# Beam Pairing Notebook

In [14]:
import h5py
import simplekml
import pandas
import geopandas as gpd
import matplotlib.pyplot as plt
from osgeo import ogr
import fiona

# Packages for kml2df (temporary addition for testing)
import os
import numpy as np
import pandas as pd
import re
import datetime

In [15]:
def kml2df(kml_filename):

    with open(kml_filename, 'r') as file:
        kml_string = file.read()
    kml_string_line = kml_string[kml_string.find('LineString_kml'):kml_string.find('</LineString>')]
    kml_coords_str = '[[' + kml_string_line[kml_string_line.find('<coordinates>')+len('<coordinates>'):kml_string_line.find('</coordinates>')-1] + ']]'
    kml_coords_str = kml_coords_str.replace(' ', '],[')
    kml_coords_array = np.array(eval(kml_coords_str))

    starts = [m.start() for m in re.finditer('<Point id=', kml_string)]
    ends = [m.start()+100 for m in re.finditer('</Point>', kml_string)]
    stringlist = [kml_string[s:e] for s,e in list(zip(starts, ends))]
    lats, lons, dts_utc, dts_local, dtstmp_utc, dtstmp_local, str_utc, str_local = [], [], [], [], [], [], [], []
    for substr in stringlist:
        coordstr = substr[substr.find('<coordinates>')+len('<coordinates>'):substr.find('</coordinates>')]
        coordstr = '['+coordstr[:coordstr.rfind(',')-1]+']'
        coords = eval(coordstr)
        lat = coords[1]
        lon = coords[0]
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        dt_str = descr[descr.find('DOY-'):]
        dt_str = dt_str[dt_str.find(' ')+1:]
        dt_str
        descr = substr[substr.find('<name>')+len('<name>'):substr.find('</name>')]
        dt_str = descr[descr.find('DOY-'):]
        dt_str = dt_str[dt_str.find(' ')+1:]
        day = int(dt_str[:dt_str.find('-')])
        month_abbr = dt_str[dt_str.find('-')+1:dt_str.rfind('-')]
        year = int(dt_str[dt_str.rfind('-')+1:dt_str.find(' ')])
        hrs = int(dt_str[dt_str.find(' ')+1:dt_str.find(':')])
        mins = int(dt_str[dt_str.find(':')+1:dt_str.rfind(':')])
        secs = int(dt_str[dt_str.rfind(':')+1:])
        datetime_str = '%4i-%3s-%02iT%02i:%02i:%02iZ' % (year, month_abbr, day, hrs, mins, secs)
        dt = datetime.datetime.strptime(datetime_str,'%Y-%b-%dT%H:%M:%SZ')
        timestamp_utc = datetime.datetime.timestamp(dt)
        lats.append(lat)
        lons.append(lon)
        dtstmp_utc.append(timestamp_utc)
    df = pd.DataFrame({'lat':lats, 'lon':lons, 'dtstmp_UTC': dtstmp_utc})
    return df

In [16]:
kml_dir = '/home/jovyan/shared-public/ICESat-2-Hackweek/KMLs'
kml_filelist = [kml_dir+'/'+f for f in os.listdir(kml_dir) if os.path.isfile(os.path.join(kml_dir, f))]
cycle = 21
cycle_search = '_cycle%i_' % cycle
cycle_filelist = [f for f in kml_filelist if ((cycle_search in f) & (f[len(kml_dir)+1] != '.'))]
nfiles = len(cycle_filelist)
print('cycle %2i: %4i KMLs' % (cycle, nfiles))

cycle 21: 1387 KMLs


In [17]:
df_list = []
startTime = datetime.datetime.now()
for i,fn in enumerate(cycle_filelist):
    df_list.append(kml2df(fn))
    print('%4i / %4i' % (i, nfiles), end='\r')
df_all = pd.concat(df_list)
print(datetime.datetime.now() - startTime)

0:02:08.498876


In [18]:
df_all['time_str'] = [datetime.datetime.strftime(datetime.datetime.fromtimestamp(t), '%a %Y-%b-%d %H:%M:%S') for t in df_all.dtstmp_UTC]
df_all

,lat,lon,dtstmp_UTC,time_str
0,0.030358,57.487437,1.700939e+09,Sat 2023-Nov-25 18:59:01
1,3.871154,57.103318,1.700939e+09,Sat 2023-Nov-25 19:00:01
2,7.712184,56.717966,1.700939e+09,Sat 2023-Nov-25 19:01:01
3,11.552985,56.330143,1.700939e+09,Sat 2023-Nov-25 19:02:01
4,15.393106,55.938517,1.700939e+09,Sat 2023-Nov-25 19:03:01
...,...,...,...,...
90,-16.409800,-101.771708,1.696407e+09,Wed 2023-Oct-04 08:08:41
91,-12.575306,-102.164307,1.696407e+09,Wed 2023-Oct-04 08:09:41
92,-8.738630,-102.552793,1.696407e+09,Wed 2023-Oct-04 08:10:41
93,-4.900199,-102.938524,1.696407e+09,Wed 2023-Oct-04 08:11:41


In [21]:
def make_new_rgts(kml_gpd):
    # Estimate location of left and right beams,
    # assuming a beam pair spacing of 3.3 km
    dellon = 3300./(np.cos(np.radians(kml_gpd['lat']))*111000)
    lon_right = kml_gpd['lon'] + dellon
    lon_left = kml_gpd['lon'] - dellon
    
    # Add left and right beams into GeoDataFrame
    right_gpd = pd.DataFrame(data={'lat': kml_gpd['lat'],
                                   'lon': lon_right})
    left_gpd = pd.DataFrame(data={'lat': kml_gpd['lat'],
                                  'lon': lon_left})
    
    return right_gpd, left_gpd

In [22]:
right, left = make_new_rgts(df_all)

In [24]:
left

,lat,lon
0,0.030358,57.457707
1,3.871154,57.073520
2,7.712184,56.687965
3,11.552985,56.299799
4,15.393106,55.907681
...,...,...
90,-16.409800,-101.802700
91,-12.575306,-102.194767
92,-8.738630,-102.582872
93,-4.900199,-102.968362
